In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = de5d43e
molecule = {'h2o': 'atmpro', 'n2o': 3.2e-07}
band = [9]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0]
  ng_refs = [7]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(2, 1.6, 1.6, 1.6, 1.6, 1.6, 1.8)]
  wgt = [(0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.9)]
o n2o band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 3.2e-07
  dv = 0.001
  klin = 2.22e-20
  molecule = n2o
  ng_adju = [0, 0]
  ng_refs = [2, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(1.8, 1.66), (1.6, 1.8)]
  wgt = [(0.6, 0.5), (0.6, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.039931                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.011983
0.000750      2.0 -0.039679                  2.0 -0.011846
0.001052      3.0 -0.039110                  3.0 -0.011469
0.001476      4.0 -0.038496                  4.0 -0.011092
0.002070      5.0 -0.037876                  5.0 -0.010710
0.002904      6.0 -0.037260                  6.0 -0.010321
0.004074      7.0 -0.036675                  7.0 -0.009922
0.005714      8.0 -0.036096                  8.0 -0.009520
0.008015      9.0 -0.035557                  9.0 -0.009105
0.011243     10.0 -0.035064                 10.0 -0.008690
0.015771     11.0 -0.034642                 11.0 -0.008262
0.022122     12.0 -0.034326                 12.0 -0.007833
0.031031     13.0 -0.034164                 13.0 -0.007398
0.043528     14.0 -0.034170                 14.0 -0.006935
0.061057     15.0 -0.032778                 15.0 -0.005463
0.085645     16.0 -0.026665                 16.0 -0.000943
0.120136     17.0 -0.015456                 17.0  0.006911
0.168516     18.0 -0.001126                 18.0  0.017073
0.236378     19.0  0.016691                 19.0  0.029900
0.331549     20.0  0.038744                 20.0  0.045976
0.465100     21.0  0.065973                 21.0  0.065997
0.652400     22.0  0.099681                 22.0  0.090685
0.915100     23.0  0.130718                 23.0  0.113919
1.283650     24.0  0.127596                 24.0  0.115519
1.800600     25.0  0.094625                 25.0  0.096190
2.525700     26.0  0.063201                 26.0  0.074115
3.542800     27.0  0.040389                 27.0  0.054388
4.969550     28.0  0.025535                 28.0  0.037982
6.970850     29.0  0.016884                 29.0  0.025234
9.778100     30.0  0.011843                 30.0  0.015646
13.715850    31.0  0.008113                 31.0  0.008340
19.239350    32.0  0.004821                 32.0  0.002924
26.987250    33.0  0.002450                 33.0 -0.000561
37.855300    34.0  0.000665                 34.0 -0.003023
53.100050    35.0 -0.001730                 35.0 -0.005942
73.887500    36.0 -0.004878                 36.0 -0.009619
97.662500    37.0 -0.007680                 37.0 -0.012968
121.437500   38.0 -0.010063                 38.0 -0.015709
145.212500   39.0 -0.013016                 39.0 -0.018430
168.987500   40.0 -0.015923                 40.0 -0.020637
192.762500   41.0 -0.016836                 41.0 -0.021103
216.537500   42.0 -0.014859                 42.0 -0.019228
240.312500   43.0 -0.010952                 43.0 -0.015485
264.087500   44.0 -0.005721                 44.0 -0.010086
287.862500   45.0  0.000363                 45.0 -0.002792
311.637500   46.0  0.005393                 46.0  0.002291
335.412500   47.0  0.011259                 47.0  0.007301
359.187500   48.0  0.018639                 48.0  0.014641
382.962500   49.0  0.027267                 49.0  0.023994
406.737500   50.0  0.036910                 50.0  0.034018
430.512500   51.0  0.047207                 51.0  0.043791
454.287500   52.0  0.056930                 52.0  0.052374
478.062500   53.0  0.066613                 53.0  0.061375
501.837500   54.0  0.076612                 54.0  0.071723
525.612500   55.0  0.086810                 55.0  0.083482
549.387500   56.0  0.097175                 56.0  0.095850
573.162500   57.0  0.107891                 57.0  0.108124
596.937500   58.0  0.122685                 58.0  0.123565
620.712500   59.0  0.135704                 59.0  0.136166
644.487500   60.0  0.145998                 60.0  0.145491
668.262500   61.0  0.153957                 61.0  0.152664
692.037500   62.0  0.159869                 62.0  0.159029
715.812500   63.0  0.164290                 63.0  0.165761
739.587500   64.0  0.167772                 64.0  0.173363
76

# Fluxes by Layer

CRD                           CLIRAD                \
                         flug       fldg      fnetg       flug          fldg   
pressure     level                                                             
0.000000e+00 1     -14.443847   0.000000 -14.443847        NaN           NaN   
1.000000e-08 1            NaN        NaN        NaN -14.715277  8.942664e-08   
6.244000e-04 2     -14.443851   0.000001 -14.443850 -14.715279  6.871865e-07   
8.759000e-04 3     -14.443852   0.000001 -14.443851 -14.715279  9.313809e-07   
1.228600e-03 4     -14.443855   0.000002 -14.443853 -14.715280  1.282171e-06   
1.723400e-03 5     -14.443858   0.000003 -14.443855 -14.715281  1.785283e-06   
2.417400e-03 6     -14.443862   0.000004 -14.443858 -14.715283  2.506785e-06   
3.390900e-03 7     -14.443868   0.000006 -14.443862 -14.715285  3.541775e-06   
4.756500e-03 8     -14.443877   0.000009 -14.443868 -14.715288  5.026918e-06   
6.672000e-03 9     -14.443889   0.000012 -14.443876 -14.715293  7.157607e-06   
9.358900e-03 10    -14.443905   0.000018 -14.443888 -14.715299  1.021529e-05   
1.312780e-02 11    -14.443928   0.000025 -14.443903 -14.715307  1.460204e-05   
1.841450e-02 12    -14.443961   0.000036 -14.443925 -14.715318  2.089689e-05   
2.583020e-02 13    -14.444006   0.000051 -14.443955 -14.715334  2.992586e-05   
3.623230e-02 14    -14.444070   0.000073 -14.443997 -14.715356  4.287678e-05   
5.082340e-02 15    -14.444160   0.000104 -14.444056 -14.715387  6.147500e-05   
7.129060e-02 16    -14.444286   0.000150 -14.444136 -14.715428  8.946435e-05   
1.000000e-01 17    -14.444453   0.000227 -14.444227 -14.715479  1.368020e-04   
1.402710e-01 18    -14.444665   0.000364 -14.444300 -14.715532  2.227057e-04   
1.967600e-01 19    -14.444917   0.000610 -14.444308 -14.715573  3.782852e-04   
2.759970e-01 20    -14.445192   0.001041 -14.444151 -14.715572  6.580357e-04   
3.871000e-01 21    -14.445432   0.001791 -14.443641 -14.715466  1.157147e-03   
5.431000e-01 22    -14.445499   0.003077 -14.442422 -14.715131  2.042347e-03   
7.617000e-01 23    -14.445085   0.005245 -14.439841 -14.714337  3.596032e-03   
1.068500e+00 24    -14.443736   0.008646 -14.435090 -14.712774  6.173676e-03   
1.498800e+00 25    -14.441492   0.012906 -14.428586 -14.710472  9.760558e-03   
2.102400e+00 26    -14.438921   0.017101 -14.421821 -14.707800  1.396714e-02   
2.949000e+00 27    -14.436283   0.020800 -14.415482 -14.704953  1.855323e-02   
4.136600e+00 28    -14.433700   0.023899 -14.409800 -14.702058  2.331073e-02   
5.802500e+00 29    -14.431279   0.026517 -14.404761 -14.699253  2.800128e-02   
8.139200e+00 30    -14.429087   0.029000 -14.400088 -14.696686  3.242014e-02   
1.141700e+01 31    -14.427244   0.031755 -14.395489 -14.694615  3.642481e-02   
1.601470e+01 32    -14.426142   0.035071 -14.391071 -14.693634  3.998679e-02   
2.246400e+01 33    -14.426596   0.039209 -14.387388 -14.694814  4.340090e-02   
3.151050e+01 34    -14.429595   0.044833 -14.384762 -14.699699  4.768465e-02   
4.420010e+01 35    -14.436584   0.052821 -14.383762 -14.710828  5.426891e-02   
6.200000e+01 36    -14.451026   0.063616 -14.387410 -14.733272  6.418290e-02   
8.577500e+01 37    -14.478387   0.077240 -14.401147 -14.773789  7.760552e-02   
1.095500e+02 38    -14.513883   0.091107 -14.422776 -14.824105  9.139675e-02   
1.333250e+02 39    -14.556998   0.105882 -14.451116 -14.882815  1.058604e-01   
1.571000e+02 40    -14.608831   0.121059 -14.487772 -14.949580  1.207159e-01   
1.808750e+02 41    -14.670144   0.137526 -14.532617 -15.023587  1.365962e-01   
2.046500e+02 42    -14.739872   0.159838 -14.580033 -15.103361  1.569324e-01   
2.284250e+02 43    -14.818964   0.197082 -14.621882 -15.190399  1.898140e-01   
2.522000e+02 44    -14.909913   0.257186 -14.652727 -15.287269  2.430695e-01   
2.759750e+02 45    -15.017081   0.348241 -14.668840 -15.398051  3.254425e-01   
2.997500e+02 46    -15.146295   0.478476 -14.667819 -15.528341  4.478672e-01   
3.235250e+02 47    -15.293

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')